In [3]:
from nba_api.stats.endpoints import playercareerstats
import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType
from nba_api.stats.library.parameters import SeasonTypePlayoffs

import json


In [4]:
def json_to_file(f_name, sometext):
    json_object = json.dumps(sometext, indent=4)
    with open(f'data/{f_name}.json', "w") as outfile:
        outfile.write(json_object)


In [5]:
nba_teams = teams.get_teams()

json_to_file("nba_teams",nba_teams)

In [74]:
team_abb_collection = []
for i in teams.get_teams():
    team_abb_collection.append(i['abbreviation'])

# team_abb_collection

In [9]:
# export player to file
all_player_list = players.get_players()
json_to_file("all_player",all_player_list)

In [10]:
# Create Map for player and playerID

player_id_map = {}
for i in all_player_list:
    player_id_map[i['id']] = i['full_name']
    # break
# player_id_map

In [34]:
player_list = []

In [71]:
# Search for player with name
player_name = "Jalen"
player_result = []
for i in all_player_list:
    if i['first_name'] == player_name:
        player_result.append(i)
    elif i['last_name'] == player_name:
        player_result.append(i)

for i in player_result:
    player_list.append(i['id'])



In [58]:
# Get player career stats

# Check for duplication
player_list = list(set(player_list))

total_df = []
for i in player_list:
    player_career = playercareerstats.PlayerCareerStats(player_id =  str(i)) 
    career_df = player_career.get_data_frames()[0]
    total_df.append(career_df)

combineCareer = pd.concat(total_df)

combineCareer

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630592,2023-24,00,1610612751,BKN,23.0,43,3,665.0,76,...,0.826,47,84,131,45,11,4,19,48,214
0,1631105,2022-23,00,1610612765,DET,19.0,67,31,1670.0,258,...,0.611,229,366,595,75,44,59,92,182,612
1,1631105,2023-24,00,1610612765,DET,20.0,61,60,1777.0,349,...,0.790,192,517,709,147,33,47,129,192,841
0,1629665,2019-20,00,1610612756,PHX,20.0,5,0,32.0,4,...,1.000,0,2,2,2,0,0,2,3,10
1,1629665,2020-21,00,1610612754,IND,21.0,4,0,12.0,1,...,1.000,2,3,5,2,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335.0,565,...,0.817,166,545,711,512,85,44,226,222,1456
5,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488.0,732,...,0.868,205,575,780,599,95,48,222,192,1898
6,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476.0,764,...,0.810,206,813,1019,584,109,63,281,191,2004
7,203999,2022-23,00,1610612743,DEN,28.0,69,69,2323.0,646,...,0.822,167,650,817,678,87,47,247,174,1690


In [70]:
# Add player name to the stat and export it to a csv file
combineCareer['PLAYER_NAME'] = combineCareer['PLAYER_ID'].map(player_id_map)

combineCareer_col = combineCareer.columns.tolist()

if 'PLAYER_NAME' not in combineCareer_col[:-1]:
    combineCareer_col = combineCareer_col[-1:] + combineCareer_col[:-1]

combineCareer=combineCareer[combineCareer_col]
combineCareer.to_csv('data/palyer_Stat.csv', sep=',',  index=False)

combineCareer


,PLAYER_NAME,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,Jalen Wilson,1630592,2023-24,00,1610612751,BKN,23.0,43,3,665.0,...,0.826,47,84,131,45,11,4,19,48,214
0,Jalen Duren,1631105,2022-23,00,1610612765,DET,19.0,67,31,1670.0,...,0.611,229,366,595,75,44,59,92,182,612
1,Jalen Duren,1631105,2023-24,00,1610612765,DET,20.0,61,60,1777.0,...,0.790,192,517,709,147,33,47,129,192,841
0,Jalen Lecque,1629665,2019-20,00,1610612756,PHX,20.0,5,0,32.0,...,1.000,0,2,2,2,0,0,2,3,10
1,Jalen Lecque,1629665,2020-21,00,1610612754,IND,21.0,4,0,12.0,...,1.000,2,3,5,2,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,Nikola Jokic,203999,2019-20,00,1610612743,DEN,25.0,73,73,2335.0,...,0.817,166,545,711,512,85,44,226,222,1456
5,Nikola Jokic,203999,2020-21,00,1610612743,DEN,26.0,72,72,2488.0,...,0.868,205,575,780,599,95,48,222,192,1898
6,Nikola Jokic,203999,2021-22,00,1610612743,DEN,27.0,74,74,2476.0,...,0.810,206,813,1019,584,109,63,281,191,2004
7,Nikola Jokic,203999,2022-23,00,1610612743,DEN,28.0,69,69,2323.0,...,0.822,167,650,817,678,87,47,247,174,1690


In [81]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_select['id'],
                            # season_nullable="20-21",
                            # season="20-21",
                            season_type_nullable=SeasonType.regular
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                              )  

games_dict = gamefinder.get_normalized_dict()
# print(SeasonTypePlayoffs.playoffs)
len(games_dict['LeagueGameFinderResults'])


KeyError: 'resultSet'

In [76]:
match_up_team = "BOS"
matchup_result = []
for i in games_dict['LeagueGameFinderResults']:
    if match_up_team in i['MATCHUP']:
    # if match_up_team.MATCH.str.contsins('MIN:
        matchup_result.append(i)

json_to_file("match_result",matchup_result)

        

In [13]:
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')
print(game)

{'SEASON_ID': '22023', 'TEAM_ID': 1610612742, 'TEAM_ABBREVIATION': 'DAL', 'TEAM_NAME': 'Dallas Mavericks', 'GAME_ID': '0022301196', 'GAME_DATE': '2024-04-14', 'MATCHUP': 'DAL @ OKC', 'WL': 'L', 'MIN': 241, 'PTS': 86, 'FGM': 32, 'FGA': 97, 'FG_PCT': 0.33, 'FG3M': 8, 'FG3A': 39, 'FG3_PCT': 0.205, 'FTM': 14, 'FTA': 17, 'FT_PCT': 0.824, 'OREB': 12, 'DREB': 29, 'REB': 41, 'AST': 19, 'STL': 8, 'BLK': 2, 'TOV': 14, 'PF': 15, 'PLUS_MINUS': -49.0}


In [14]:
# Get Playbyplay
from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2('0022301196')
pbp = pbp.get_data_frames()[0]
# pbp.head()

In [15]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022301196') 

# box.away_team_player_stats.get_dict()  

# Data Sets
# box.game.get_dict()                  #equal to box.get_dict()['game']
#box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
#box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
#box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
#box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
#box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
#box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
#box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
#box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [7]:
# Select the dictionary for the Team (BOS, LAL, IND), which contains their team ID 
team_abb = 'DAL'
team_select = [team for team in nba_teams if team['abbreviation'] == team_abb][0]
team_select
# f'team_id: {team_select["id"]}'


{'id': 1610612742,
 'full_name': 'Dallas Mavericks',
 'abbreviation': 'DAL',
 'nickname': 'Mavericks',
 'city': 'Dallas',
 'state': 'Texas',
 'year_founded': 1980}

In [75]:
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_select['id'],
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular
                            # season_type_nullable=SeasonTypePlayoffs.playoffs
                                              )  

games_dict = gamefinder.get_normalized_dict()
# print(SeasonTypePlayoffs.playoffs)
# games_dict


In [17]:
# gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable="1610612742")

games = gamefinder.get_data_frames()[0]


total_game = games.groupby(games.SEASON_ID.str[-4:])[['GAME_ID']].count().loc['2015':]
total_game

,GAME_ID
SEASON_ID,
2023,82
